# <p style="background-color:coral;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 20px;"><b>Housing Prices Prediction with Random Forest</b></p>
![](https://img.freepik.com/free-vector/modern-cottage-houses-set_74855-305.jpg?t=st=1658400642~exp=1658401242~hmac=d9fa26ceb8482ae8408d2e9b4e1d8b3315e530babfcee2245ad8b54eab9690cf&w=996)

<b>Hi guys </b>😀

In this notebook, I'm going to show you how to perform random forest using housing prices dataset.

<b>Table of contents:</b>
<ul>
<li><a href="#Loading">Loading the dataset</a></li>  
<li><a href="#Understanding">Understanding the dataset</a></li>         
<li><a href="#Data-Preprocessing">Data preprocessing</a></li>
<li><a href="#Missing">Handling missing data</a></li>
<li><a href="#Splitting">Splitting the Dataset</a></li>
<li><a href="#Pipelines">Pipelines for data preprocessing</a></li>
<li><a href="#Model-Building">Model building</a></li>
<li><a href="#Cross-Validation">Cross-validation</a></li>      
<li><a href="#Grid-Search">Grid Search</a></li>        
<li><a href="#Conclusion">Conclusion</a></li>   
</ul>

Happy learning 🐱‍🏍 

<a id="Loading"></a>
# <p style="background-color:coral;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 20px;"><b>Loading the Dataset</b></p>

The dataset I'm going to load is the housing prices dataset. The dataset includes the train and test set. Let's read these datasets with the `read_csv` method and then look at the first five rows with the `head` method.

In [1]:
import pandas as pd
df_train = pd.read_csv("../input/home-data-for-ml-course/train.csv")
df_test = pd.read_csv("../input/home-data-for-ml-course/test.csv")
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


<a id="Understanding"></a>
# <p style="background-color:coral;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 20px;"><b>Understanding the Dataset</b></p>

Let's take a look at the shape of train and test set with the `shape` attribute.

In [2]:
print("The shape of train set: ", df_train.shape)
print("The shape of test set: ", df_test.shape)

The shape of train set:  (1460, 81)
The shape of test set:  (1459, 80)


Let's have a look at the column types with the `dtypes` attribute.

In [3]:
df_train.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

You can also use the `info` method to see information such as the index dtype and columns, non-null values and memory usage.

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

Let's see the summary statistics of numerical columns with the `describe` method.

In [5]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,1460.0,730.500000,421.610009,1.0,365.75,730.5,1095.25,1460.0
MSSubClass,1460.0,56.897260,42.300571,20.0,20.00,50.0,70.00,190.0
LotFrontage,1201.0,70.049958,24.284752,21.0,59.00,69.0,80.00,313.0
LotArea,1460.0,10516.828082,9981.264932,1300.0,7553.50,9478.5,11601.50,215245.0
OverallQual,1460.0,6.099315,1.382997,1.0,5.00,6.0,7.00,10.0
OverallCond,1460.0,5.575342,1.112799,1.0,5.00,5.0,6.00,9.0
YearBuilt,1460.0,1971.267808,30.202904,1872.0,1954.00,1973.0,2000.00,2010.0
YearRemodAdd,1460.0,1984.865753,20.645407,1950.0,1967.00,1994.0,2004.00,2010.0
MasVnrArea,1452.0,103.685262,181.066207,0.0,0.00,0.0,166.00,1600.0
BsmtFinSF1,1460.0,443.639726,456.098091,0.0,0.00,383.5,712.25,5644.0


<a id="Data-Preprocessing"></a>
# <p style="background-color:coral;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 20px;"><b>Data Preprocessing</b></p>

The first column is the `Id`. Let's convert this column into the index.

In [6]:
df_train.set_index("Id", inplace=True)
df_test.set_index("Id", inplace=True)
df_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## <span style="color:Orange">Handling Missing Data</span>


Let's take a look at missing data in each column with the `isnull` method.

In [7]:
df_train.isnull().sum()

MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
Street             0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 80, dtype: int64

Since there are many columns in the dataset, we can't see the number of missing data in all columns. Let's sort the columns with the most missing data using the `sort_values` method and look at the first twenty rows.

In [8]:
cols_with_null = df_train.isnull().sum().sort_values(ascending=False)
cols_with_null.head(20)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtExposure      38
BsmtFinType2      38
BsmtCond          37
BsmtQual          37
BsmtFinType1      37
MasVnrArea         8
MasVnrType         8
Electrical         1
MSSubClass         0
dtype: int64

To see the count of all missing data in the dataset, let me use the `sum` method one more time.

In [9]:
print("Total number of missing data in the dataset: ", df_train.isnull().sum().sum())

Total number of missing data in the dataset:  6965


Let's look at the number of missing data in the Sales Price target variable.

In [10]:
df_train["SalePrice"].isnull().sum()

0

Let's remove the first six columns with the most missing data with the `drop` method.

In [11]:
cols_to_drop = (cols_with_null.head(6).index).tolist()
df_train.drop(cols_to_drop, axis=1, inplace=True)
df_test.drop(cols_to_drop, axis=1, inplace=True)

## <span style="color:Orange">Creating the target and feature variables</span>

The SalePrice column is the target variable and the other columns is features. Let's assign y and X variables to these columns, respectively.

In [12]:
y = df_train.SalePrice
X = df_train.drop(["SalePrice"], axis=1)

## <span style="color:Orange">Splitting the dataset</span>

Let's split the dataset into the train and test set with the `train_test_split` method.

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y,train_size=0.8, random_state=0)

## <span style="color:Orange">Handling the categorical and numerical columns</span>

Data preprocessing is different for categorical and numeric columns. Let's select categorical and numeric columns. I'm going to remove columns with more than ten subcategories.

In [14]:
categorical_cols=[cname for cname in X_train.columns 
                  if X_train[cname].nunique()<10 and X_train[cname].dtype == "object"]

In [15]:
numerical_cols=[cname for cname in X_train.columns 
                if X_train[cname].dtype in ["int64", "float64"]]

Let's have a look at the number of categorical and numerical columns.

In [16]:
print("The number of categorical columns: ", len(categorical_cols))
print("The number of numerical columns: ", len(numerical_cols))

The number of categorical columns:  35
The number of numerical columns:  35


We've selected `70(=35+35)` columns. Let's remove any other columns we didn't select from the datasets.

In [17]:
my_cols=categorical_cols+numerical_cols
X_train = X_train[my_cols]
X_val = X_val[my_cols]
X_test = df_test[my_cols]

## <span style="color:Orange">Pipelines for data preprocessing</span>

A machine learning pipeline allows us to combine a series of steps involved in training a model. Let's import the necessary libraries to build the pipelines.

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

Let's build a pipeline for numerical columns to handle missing data and scale data.

In [19]:
numerical_transformer = Pipeline(steps=[
    ("imputer_num", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

Let's build an other pipeline for categorical columns to handle missing data and perform one-hot encoding.

In [20]:
categorical_transformer = Pipeline(steps = [
    ("imputer_cal", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

Let's apply these transformers to categorical and numerical columns.

In [21]:
preprocessor = ColumnTransformer(transformers=[
    ("num", numerical_transformer, numerical_cols),
    ("cat", categorical_transformer, categorical_cols)
])

<a id="Model-Building"></a>
# <p style="background-color:coral;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 20px;"><b>Model Building</b></p>

Random forests is an ensemble learning method used for classification and regression. Let's build a simple random forest model.

In [22]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=0)

Let's create a pipeline for data preprocessing and model building steps.

In [23]:
my_pipeline = Pipeline(steps=[ ("preprocessor", preprocessor),("model", rf)])

Let's train the model with the `train` method.

In [24]:
my_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer_num',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['MSSubClass', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'BsmtUnfSF',
                                                   'TotalBsmtSF', '1stFlrSF',
                                                   '2ndFlrSF', 'LowQualFinSF',
          

## <span style="color:Orange">Model evaluation</span>

Let's predict the validation data with the `predict` method.

In [25]:
val_preds = my_pipeline.predict(X_val)

Let's see the performance of model on validation data with the `mean_absulate_error` function.

In [26]:
from sklearn.metrics import mean_absolute_error
print("Validation MAE: ", mean_absolute_error(y_val, val_preds))

Validation MAE:  17721.242602739727


<a id="Cross-Validation"></a>
# <p style="background-color:coral;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 20px;"><b>Cross-Validation</b></p>

Cross-validation is a resampling method that allows us to use different portions of the data to test and train a model on different iterations. Let's find the cross validation score with the `cross_val_score` function and calculate the mean of cross validation scores with the `mean` method.

In [27]:
from sklearn.model_selection import cross_val_score
scores = -1 * cross_val_score(my_pipeline, X,y, cv = 5, scoring="neg_mean_absolute_error")
print("Mean Cross Validation Score: ", scores.mean())

Mean Cross Validation Score:  17568.15694520548


<a id="Grid-Search"></a>
# <p style="background-color:coral;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 20px;"><b>Grid Search</b></p>

The grid search allows us to generate candidates from the grid of parameter values specified by the param_grid parameter. Let's find the best hyperparameters of random forest model with the `GridSearchCV` class.

In [28]:
from sklearn.model_selection import GridSearchCV
param_grid = { 
    'model__n_estimators': [500, 600, 700],
    'model__max_features': ['auto','sqrt','log2'],
    'model__max_depth' : [5,6,7],
    'model__criterion' :['squared_error','absolute_error','poisson']}
GridCV = GridSearchCV(my_pipeline, param_grid, n_jobs= -1)
GridCV.fit(X_train,y_train)  
print(GridCV.best_params_)    
print(GridCV.best_score_)

{'model__criterion': 'squared_error', 'model__max_depth': 7, 'model__max_features': 'auto', 'model__n_estimators': 500}
0.851573621723715


Let's predict the test data with the `predict` method.

In [29]:
preds_test = GridCV.predict(X_test)

Let's convert these predictions into a dataframe.

In [30]:
output = pd.DataFrame({'Id': X_test.index, 'SalePrice': preds_test})
output.head()

,Id,SalePrice
0,1461,125762.249652
1,1462,151586.396654
2,1463,181441.565045
3,1464,181278.359674
4,1465,204940.884297


Let's write this dataframe to an `csv` file.

In [31]:
output.to_csv('submission.csv', index=False)

You can now submit this file to the competition!

<a id="Conclusion"></a>
# <p style="background-color:coral;font-family:newtimeroman;font-size:150%;color:white;text-align:center;border-radius:20px 20px;"><b>Conclusion</b></p>

### That's it. In this notebook, I first performed EDA and then built a random forest model to predict house prices. I also used the grid search technique to find the combination of best hyperparameters.

### Thanks for reading 😀 If you like this notebook, please upvote it 😊

### Don't forget to follow us on [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [Twitter](http://twitter.com/tirendazacademy) | [GitHub](http://github.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) | [Kaggle](https://www.kaggle.com/tirendazacademy)